In [1]:
import numpy as np
import random

np.set_printoptions(precision=4)

In [2]:
L = 7
N = 14
mu = [1, 1.5, 1.3, 1.7, 1.9, 1.4, 2]
Theta = np.array([
    [0, 0.3, 0, 0.7, 0, 0, 0],
    [0, 0, 0.6, 0, 0, 0.4, 0],
    [0.5, 0, 0, 0.5, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0.3, 0, 0, 0.7, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0.8, 0, 0, 0, 0.2, 0, 0],
])

In [3]:
def check_list(l):
    eps = 0.1
    for i in range(1, len(l)):
        if abs(l[0] - l[i]) > eps:
            return False
    return True

### Маршрутная матрица задается произвольно, но чтобы не было систем, являющихся источниками или стоками. Необходимо подобрать такой вектор интенсивностей обслуживания, чтобы м.о. числа требований в системах были одинаковы.

In [8]:
def get_s(L, N, mu, Theta):
    omega = np.array([1 / L for _ in range(L)])
    for _ in range(1000000):
        omega = omega.dot(Theta)
    
    # М.о. числа требований в системах
    s = [[0] * L for _ in range(N + 1)]
    # М.о. длительности пребывания требований в системах
    u = [[0] * L for _ in range(N + 1)]

    for Y in range(1, N + 1):
        for i in range(L):
            u[Y][i] = 1 / mu[i] * (s[Y-1][i] + 1)
        for i in range(L):
            summa = 0
            for j in range(L):
                summa += omega[j] * u[Y][j]
            s[Y][i] = omega[i] * u[Y][i] * Y / summa

    return s[N]

copy_mu = [0.1] * L
count = 0
s = None

while True:
    s = get_s(L, N, copy_mu, Theta)
    if check_list(s):
        break

    s_max = max(s)
    s_min = min(s)
    i = s.index(s_min)
    j = s.index(s_max)

    count += 1
    if count % 50 == 0:
        i = random.randint(0, len(s) - 1)
        j = random.randint(0, len(s) - 1)
        if s[i] > s[j]:
            i, j = j, i
        s_min = s[i]
        s_max = s[j]

    print('Перенос интенсивности обслуживания из системы', i+1 , 'в систему', j+1)
    print(s[i], '->', s[j])

    delta = min(copy_mu[i], copy_mu[j])
    gamma = random.random() * delta * (s_max - s_min) / s_max
    copy_mu[i] -= gamma
    copy_mu[j] += gamma

print('\n', '*' * 100)
print('mu при котором достигается равенство м.о. числа требований в системах:\n', copy_mu)
print('М.о. числа требований в системах:\n', s)

Перенос интенсивности обслуживания из системы 2 в систему 5
0.2929480602440083 -> 4.0831801233070575
Перенос интенсивности обслуживания из системы 3 в систему 2
0.25248735040787634 -> 11.129275759975705
Перенос интенсивности обслуживания из системы 3 в систему 2
0.5939596055167973 -> 6.545896657799083
Перенос интенсивности обслуживания из системы 3 в систему 2
0.6605306903306769 -> 5.67249072845809
Перенос интенсивности обслуживания из системы 5 в систему 2
0.8509737007261045 -> 3.029105552089234
Перенос интенсивности обслуживания из системы 3 в систему 1
1.003371660341923 -> 3.09675887716381
Перенос интенсивности обслуживания из системы 5 в систему 4
1.1098910215769073 -> 2.858712396798744
Перенос интенсивности обслуживания из системы 4 в систему 6
1.2924205235344852 -> 2.678368233687771
Перенос интенсивности обслуживания из системы 2 в систему 7
1.3696198313635504 -> 2.7333902630203464
Перенос интенсивности обслуживания из системы 6 в систему 2
1.3381042379770214 -> 2.454131541259234

### Вектор интенсивностей обслуживания задается произвольно. Необходимо подобрать такую маршрутную матрицу (при сохранении топологии), чтобы коэффициенты использования приборов систем были одинаковы.

In [13]:
def get_psy(L, N, mu, Theta):
    omega = np.array([1 / L for _ in range(L)])
    for _ in range(1000000):
        omega = omega.dot(Theta)
        
    # М.о. числа требований в системах
    s = [[0] * L for _ in range(N + 1)]
    # М.о. длительности пребывания требований в системах
    u = [[0] * L for _ in range(N + 1)] 
    # Коэффициенты использования систем
    psy = [0] * L
    # М.о. длительности ожидания требований в очереди системы
    w = [0] * L
    # М.о. числа требований, ожидающих обслуживание в очереди системы
    b = [0] * L
    # М.о. числа занятых приборов в системах
    h = [0] * L
    # Интенсивность входящего потока требований в системы
    lambdas = [0] * L
    
    # рекуррентные формулы
    for Y in range(1, N + 1):
        for i in range(L):
            u[Y][i] = 1 / mu[i] * (s[Y-1][i] + 1)
        for i in range(L):
            summa = 0
            for j in range(L):
                summa += omega[j] * u[Y][j]
            s[Y][i] = omega[i] * u[Y][i] * Y / summa
            
    for i in range(L):
        w[i] = u[N][i] - (1 / mu[i])
        b[i] = s[N][i] * w[i] / u[N][i]
        h[i] = s[N][i] - b[i]
        lambdas[i] = h[i] * mu[i]
        psy[i] = lambdas[i] / mu[i]
            
    return psy




def find_psy(L, N, mu, Theta):
    for s0 in np.arange(0.1, 1, 0.1):
        for s1 in np.arange(0.1, 1, 0.1):
            for s2 in np.arange(0.1, 1, 0.1):
                for s3 in np.arange(0.1, 1, 0.1):
                    for s4 in np.arange(0.1, 1, 0.1):
                        for s5 in np.arange(0.1, 1, 0.1):
                            for s6 in np.arange(0.1, 1, 0.1):
                                Theta[0][1], Theta[0][4] = s0, 1 - s0
                                Theta[1][2], Theta[1][5] = s1, 1 - s1
                                Theta[2][3], Theta[2][6] = s2, 1 - s2
                                Theta[3][4], Theta[3][0] = s3, 1 - s3
                                Theta[4][5], Theta[4][1] = s4, 1 - s4
                                Theta[5][6], Theta[5][2] = s5, 1 - s5
                                Theta[6][0], Theta[6][3] = s6, 1 - s6
                                psy = get_psy(L, N, mu, Theta)
                                if check_list(psy):
                                    return psy

                                
                                
mu = [1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2]
Theta = np.array([
    [0, 0.3, 0, 0, 0.7, 0, 0],
    [0, 0, 0.6, 0, 0, 0.4, 0],
    [0, 0, 0, 0.5, 0, 0, 0.5],
    [0.2, 0, 0, 0, 0.8, 0, 0],
    [0, 0.3, 0, 0, 0, 0.7, 0],
    [0, 0, 0.1, 0, 0, 0, 0.9],
    [0.8, 0, 0, 0.2, 0, 0, 0],
])

psy = find_psy(L, N, mu, Theta)
print('Маршрутная матрица Theta:\n', Theta)
print()
print('при котой коэффициенты использования приборов систем одинаковы и равны:\n', psy)

Маршрутная матрица Theta:
 [[0.  0.1 0.  0.  0.9 0.  0. ]
 [0.  0.  0.1 0.  0.  0.9 0. ]
 [0.  0.  0.  0.1 0.  0.  0.9]
 [0.9 0.  0.  0.  0.1 0.  0. ]
 [0.  0.9 0.  0.  0.  0.1 0. ]
 [0.  0.  0.9 0.  0.  0.  0.1]
 [0.1 0.  0.  0.9 0.  0.  0. ]]

при котой коэффициенты использования приборов систем одинаковы и равны:
 [0.7000000000000002, 0.7000000000000002, 0.7000000000000002, 0.7000000000000002, 0.7000000000000002, 0.7000000000000002, 0.7000000000000002]
